In [21]:
from rank_bm25 import BM25L
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import re
import time 

In [4]:
test = pd.read_csv("test_df.tsv", sep="\t")

test.head()

,queries,positive,negative
0,A system for charging or maintaining a batter...,A method for battery charger and diagnosis wi...,A ductile failure anchor for a concrete eleme...
1,A method for calculating a path delay in stat...,A non transitory computer readable medium car...,A distance evaluation method for evaluating d...
2,A preventive or therapeutic agent for diabete...,A method for measuring glutamyl transpeptidas...,A communication system comprising a plurality...
3,A semiconductor device comprising a first sem...,An integrated circuit comprising a first pitc...,canceled A lighting system comprising an opti...
4,A moveable barrier operator system comprising...,A movable barrier operator having automatic f...,A remote control system which comprises a rem...


In [10]:
queries = test['queries'].values 
positives = test['positive'].values 
negatives = test['negative'].values 

corpus = np.concatenate([positives, negatives])

# BM25L Ranking

In [13]:
def simple_tok(sent): 
    cleaned = re.sub('[^A-Za-z]+', ' ', sent) 
    return cleaned.split(" ") 

In [17]:
tok_corpus = [simple_tok(s) for s in tqdm(corpus, position=0, leave=True)] 

bm25 = BM25L(tok_corpus)

100%|██████████| 77886/77886 [00:54<00:00, 1429.86it/s]


In [18]:
query = queries[0]

tokenized_query = simple_tok(query)

scores = bm25.get_scores(query)

In [19]:
best_docs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True) 

for i, b in enumerate(best_docs):
    if b == 0: 
        print(i+1)


43


# Calculate MRR for the first 100 queries

In [22]:
t0 = time.time() 
ranks = [] 
for i in range(100): 
    query = queries[i] 
    tokenized_query = simple_tok(query) 
    scores = bm25.get_scores(query) 
    best_docs = sorted(range(len(scores)), key=lambda j: scores[j], reverse=True) 

    for j, b in enumerate(best_docs):
        if b == 0: 
            ranks.append(j+1)

print(time.time() - t0)

31253.127210855484


In [23]:
mrr = 0 

for r in ranks: 
    mrr += 1/r 
    
mrr / len(ranks)

0.017716843736458866

In [24]:
np.mean(ranks)

59.36